In [3]:
import pymysql

class DatabaseManager:
    def __init__(self):
        self.conn = pymysql.connect(
            host='127.0.0.1',
            user='root',
            password='0000',
            db='market_db',
            charset='utf8'
        )
        self.cursor = self.conn.cursor()

    def execute_query(self, query):
        # 쿼리 실행
        self.cursor.execute(query)
        # 결과 가져오기
        result = self.cursor.fetchall()
        return result

    def get_column_names(self):
        # 열 이름 가져오기
        column_names = [desc[0] for desc in self.cursor.description]
        return column_names

    def close(self):
        self.cursor.close()
        self.conn.close()

# 사용 예제
db_manager = DatabaseManager()

# SQL 쿼리
query = 'SELECT * FROM member'  # 적절한 쿼리로 변경하세요

# 쿼리 실행 및 결과 가져오기
result = db_manager.execute_query(query)

result

(('APN', '에이핑크', 6, '경기', '031', '77777777', 164, datetime.date(2011, 2, 10)),
 ('BLK', '블랙핑크', 4, '경남', '055', '22222222', 163, datetime.date(2016, 8, 8)),
 ('GRL', '소녀시대', 8, '서울', '02', '44444444', 168, datetime.date(2007, 8, 2)),
 ('ITZ', '잇지', 5, '경남', None, None, 167, datetime.date(2019, 2, 12)),
 ('MMU', '마마무', 4, '전남', '061', '99999999', 166, datetime.date(2014, 6, 19)),
 ('OMY', '오마이걸', 7, '서울', None, None, 160, datetime.date(2015, 4, 21)),
 ('RED', '레드벨벳', 4, '경북', '054', '55555555', 161, datetime.date(2014, 8, 1)),
 ('SPC', '우주소녀', 13, '서울', '02', '88888888', 162, datetime.date(2016, 2, 25)),
 ('TWC', '트와이스', 9, '서울', '02', '11111111', 167, datetime.date(2015, 10, 19)),
 ('WMN', '여자친구', 6, '경기', '031', '33333333', 166, datetime.date(2015, 1, 15)))

In [1]:
import sys
import pymysql
from PyQt5.QtWidgets import *
from PyQt5 import uic

# UI 파일 로드
form_class = uic.loadUiType("pyqt6.ui")[0]

# 메인 윈도우 클래스 정의
class WindowClass(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
        # 버튼 클릭을 메서드에 연결
        self.pushButton.clicked.connect(self.execute_query)
         # 콤보 박스 선택 변경을 메서드에 연결
        self.comboBox.currentIndexChanged.connect(self.clear_table)
        self.comboBox.addItems([
            "Q1) 구매 금액이 1000원 이상인 회원 이름과 구매물품명, 구매 금액을 조회 하시오.",
            "Q2) 구매 분류가 디지털인 항목을 구매한 회원 명, 전화번호, 구매분류, 물품명, 구매금액을 조회하시오.",
            "Q3) 인원이 6명 이상인 회원이 구매한 물품, 회원명, 회원수, 구매금액을 가격 높은순으로 조회 하시오.",
            "Q4) 가장 비싼 단가를 가진 물품을 구매한 회원의 회원명, 구매단가, 구매물품, 국번전화번호, 평균키, 데뷔 일자를 구하시오. 국번전화번호의 값은 중간에 '-' 으로 연결, 나온 결과값의 정렬은 데뷔 일자가 가장 빠른 순으로 구하시오.",
            "Q5) 키가 165 이하이고 총 구매 금액이 500이 넘는 회원의 회원 아이디, 회원 키, 총구매 금액을 총 구매 수량이 많은 순으로 탑3 까지 조회",
            "Q6) 서울 출생이고 구매하지 않는 회원의 회원 아이디, 데뷔일자를 데뷔 일자가 빠른 순으로 탑2 까지 조회",
            "Q7) 아이즈원은 거제 출생이고, 폰은 없다. 평균키는 170이며 데뷔 날짜및 멤버수는 검색, ID는 IVE 이다. 아이즈 원은 청바지와 에어팟을 각 3개씩 샀다. insert 하세요.",
            "Q8) 마마무가 자신들의 평균키가 166이라고 정정해달라는 요청이 왔다. 이를 update 하시오.",
            "Q9) 멤버 이름이 4글자인 멤버의 총 평균 키를 조회하시오:",
            "Q10) 서적이 모두 환불 되었습니다. 서적을 구매한 회원 네임과 멤버의 전화번호, 환불할 총 구매 금액을 조회 하시오."
        ])
        # MySQL 연결 초기화
        self.conn = pymysql.connect(host = '127.0.0.1', user = 'root', password = '0000', db = 'market_db', charset = 'utf8')
        self.cursor = self.conn.cursor()

        layout = QVBoxLayout()
        layout.addWidget(self.comboBox)
        layout.addWidget(self.pushButton)
        layout.addWidget(self.textEdit)
        layout.addWidget(self.tableWidget)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

    def execute_query(self):
        query_number = self.comboBox.currentIndex()	  # 콤보 박스에서 선택한 SQL 쿼리 가져오기
        query = [
    """SELECT M.mem_name 회원이름, B.prod_name 구매물품명, B.price * B.amount 구매금액
        FROM buy B
            INNER JOIN member M
            ON B.mem_id = M.mem_id
        WHERE (B.price * B.amount) >=1000;""",

    """SELECT m.mem_name, concat(m.phone1, m.phone2) '전화번호', b.group_name, b.prod_name, b.price * b.amount 구매금액
        FROM buy b
            INNER JOIN member m
            ON m.mem_id = b.mem_id
        WHERE b.group_name = '디지털';""",

    """SELECT prod_name, B.mem_id, mem_number, price
        FROM buy B
            INNER JOIN member M
            ON B.mem_id = M.mem_id
        WHERE mem_number >= 6
        ORDER BY price DESC;""",

    """SELECT M.mem_id '회원아이디', M.height '회원 키', SUM(b.price*b.amount) '총구매 금액'
        FROM member M
            INNER JOIN buy B
            ON M.mem_id = B.mem_id
        GROUP BY m.mem_id
        HAVING M.height <= 165 AND SUM(B.price * b.amount) > 500
        ORDER BY sum(B.amount) DESC
        LIMIT 3;""",

    """SELECT M.mem_id, M.height, SUM(B.price * B.amount) AS "총구매 금액", SUM(B.amount) AS total_amount
        FROM buy B
            INNER JOIN member M
            ON B.mem_id = M.mem_id
        GROUP BY M.mem_id, M.height
        HAVING M.height <= 165 AND SUM(B.price * B.amount) > 500
        ORDER BY total_amount DESC
        LIMIT 3;""",

    """SELECT M.mem_id, M.debut_date
        FROM buy B
            RIGHT OUTER JOIN member M
            ON B.mem_id = M.mem_id
        WHERE amount is null and M.addr = '서울'
        ORDER BY debut_date
        LIMIT 2;""",

    """INSERT INTO member VALUES('IZO', '아이즈원', 10, '거제', NULL, NULL, 170, '2018-10-29');
    INSERT INTO buy VALUES(NULL, 'IZO', '에어팟', '디지털', 80, 3);
    INSERT INTO buy VALUES(NULL, 'IZO', '청바지', '패션', 50, 3);""",

    """UPDATE member
        SET height = 166
        WHERE mem_name = '마마무';""",

    """SELECT mem_name, AVG(height)
        FROM member
        WHERE mem_name LIKE '____'
        GROUP BY mem_id;""",

    """SELECT M.mem_name "회원 이름", CONCAT(M.phone1, '-', M.phone2) "전화번호", SUM(B.price * B.amount) "환불한 총 금액"
        FROM buy B
            INNER JOIN member M
            ON B.mem_id = M.mem_id
        WHERE group_name = '서적'
        GROUP BY M.mem_name, M.phone1, M.phone2;"""
        ]
        try:
            # SELECT 문만 실행
            if query[query_number].lower().startswith("select"):
                self.cursor.execute(query[query_number])
                self.textEdit.setText(query[query_number])                
                # 테이블 초기화
                self.tableWidget.clearContents()
                self.tableWidget.setRowCount(0)

                # 모든 결과를 가져와서 표시
                results = self.cursor.fetchall()

                # 결과를 테이블에 삽입
                if results:
                    # description - 결과 집합의 열에 대한 정보를 제공하는 튜플 목록 (column_name, type_code, display_size, internal_size, precision, scale, null_ok)
                    columns = [desc[0] for desc in self.cursor.description]
                    self.tableWidget.setColumnCount(len(columns))
                    self.tableWidget.setHorizontalHeaderLabels(columns)

                    # 행 데이터 삽입 - tuple 형태로 데이터 출력되므로 enumerate로 인덱스랑 데이터 분리
                    for row_number, row_data in enumerate(results):
                        self.tableWidget.insertRow(row_number)
                        for column_number, data in enumerate(row_data):
                            self.tableWidget.setItem(row_number, column_number, QTableWidgetItem(str(data)))
            else:
                # SELECT를 제외한 INSERT, UPDATE문 실행
                for query in query[query_number].split(';'):
                    if query.strip():
                        self.cursor.execute(query)
                self.conn.commit()
                self.tableWidget.setRowCount(1)
                self.tableWidget.setColumnCount(1)
                self.tableWidget.setItem(0, 0, QTableWidgetItem("쿼리가 성공적으로 실행되었습니다."))
        except pymysql.MySQLError as e:
            self.tableWidget.setRowCount(1)
            self.tableWidget.setColumnCount(1)
            self.tableWidget.setItem(0, 0, QTableWidgetItem(f"오류 발생: {e}"))

    def clear_table(self):
        # 테이블 초기화
        self.tableWidget.clearContents()
        self.tableWidget.setRowCount(0)
        self.tableWidget.setColumnCount(0)
        self.textEdit.clear()

    def closeEvent(self, event):
        # 창이 닫힐 때 데이터베이스 연결 닫기
        self.conn.close()
        event.accept()

if __name__ == "__main__":
    # 애플리케이션 생성
    app = QApplication(sys.argv)

    # 윈도우 인스턴스 생성
    myWindow = WindowClass()

    # 윈도우 표시
    myWindow.show()

    # 애플리케이션 이벤트 루프 실행
    app.exec_()

AttributeError: 'WindowClass' object has no attribute 'pushButton'

In [3]:
import pymysql

# 데이터베이스 연결 설정
conn = pymysql.connect(host = '127.0.0.1', user = 'root', password = '0000', db = 'market_db', charset = 'utf8')

# 커서 객체 생성
cursor = conn.cursor()

# 데이터 조회
select_query = "SELECT * FROM member;"
cursor.execute(select_query)
results = cursor.fetchall()

for i in results:
    print(i[:2])

# 데이터베이스 연결 종료
cursor.close()
conn.close()

('APN', '에이핑크')
('BLK', '블랙핑크')
('GRL', '소녀시대')
('ITZ', '잇지')
('MMU', '마마무')
('OMY', '오마이걸')
('RED', '레드벨벳')
('SPC', '우주소녀')
('TWC', '트와이스')
('WMN', '여자친구')


In [1]:
import sys
import pymysql
from PyQt5.QtWidgets import *
from PyQt5 import uic

# UI 파일 로드
form_class = uic.loadUiType("pyqt6.ui")[0]

# 메인 윈도우 클래스 정의
class WindowClass(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        
        # 버튼 클릭을 메서드에 연결
        self.executeUserQueryButton.clicked.connect(self.execute_user_query)
        self.executeComboBoxQueryButton.clicked.connect(self.execute_combobox_query)
         # 콤보 박스 선택 변경을 메서드에 연결
        self.comboBox.currentIndexChanged.connect(self.clear_table)

        # MySQL 연결 초기화
        self.conn = pymysql.connect(host = '127.0.0.1', user = 'root', password = '0000', db = 'market_db', charset = 'utf8')
        self.cursor = self.conn.cursor()
        select_query = "SELECT * FROM solve;"
        self.cursor.execute(select_query)
        self.results = self.cursor.fetchall()
        self.query = []
        for result in self.results:
            self.comboBox.addItems([result[0]])
            self.query.append(result[1])
        layout = QVBoxLayout()
        layout.addWidget(self.comboBox)
        layout.addWidget(self.executeUserQueryButton)
        layout.addWidget(self.executeComboBoxQueryButton)
        layout.addWidget(self.textEdit)
        layout.addWidget(self.tableWidget)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

    def execute_user_query(self):
        # 텍스트 에디트에 입력된 쿼리를 가져옴
        user_query = self.textEdit.toPlainText().strip()
        if user_query:
            try:
                if user_query.lower().startswith("select"):
                    self.cursor.execute(user_query)
                    self.textEdit.setText(user_query)                
                    # 테이블 초기화
                    self.tableWidget.clearContents()
                    self.tableWidget.setRowCount(0)

                    # 모든 결과를 가져와서 표시
                    results = self.cursor.fetchall()

                    # 결과를 테이블에 삽입
                    if results:
                        # description - 결과 집합의 열에 대한 정보를 제공하는 튜플 목록 (column_name, type_code, display_size, internal_size, precision, scale, null_ok)
                        columns = [desc[0] for desc in self.cursor.description]
                        self.tableWidget.setColumnCount(len(columns))
                        self.tableWidget.setHorizontalHeaderLabels(columns)

                        # 행 데이터 삽입 - tuple 형태로 데이터 출력되므로 enumerate로 인덱스랑 데이터 분리
                        for row_number, row_data in enumerate(results):
                            self.tableWidget.insertRow(row_number)
                            for column_number, data in enumerate(row_data):
                                self.tableWidget.setItem(row_number, column_number, QTableWidgetItem(str(data)))
                else:
                    for query in user_query.split(';'):
                        if query.strip():
                            self.cursor.execute(query)
                    self.conn.commit()
                    self.tableWidget.setRowCount(1)
                    self.tableWidget.setColumnCount(1)
                    self.tableWidget.setItem(0, 0, QTableWidgetItem("쿼리가 성공적으로 실행되었습니다."))
            except pymysql.MySQLError as e:
                self.tableWidget.setRowCount(1)
                self.tableWidget.setColumnCount(1)
                self.tableWidget.setItem(0, 0, QTableWidgetItem(f"오류 발생: {e}"))

    def execute_combobox_query(self):
        query_number = self.comboBox.currentIndex()	  # 콤보 박스에서 선택한 SQL 쿼리 가져오기
        try:
            if self.query[query_number].lower().startswith("select"):
                self.cursor.execute(self.query[query_number])
                self.textEdit.setText(self.query[query_number])                
                # 테이블 초기화
                self.tableWidget.clearContents()
                self.tableWidget.setRowCount(0)

                # 모든 결과를 가져와서 표시
                results = self.cursor.fetchall()

                # 결과를 테이블에 삽입
                if results:
                    # description - 결과 집합의 열에 대한 정보를 제공하는 튜플 목록 (column_name, type_code, display_size, internal_size, precision, scale, null_ok)
                    columns = [desc[0] for desc in self.cursor.description]
                    self.tableWidget.setColumnCount(len(columns))
                    self.tableWidget.setHorizontalHeaderLabels(columns)

                    # 행 데이터 삽입 - tuple 형태로 데이터 출력되므로 enumerate로 인덱스랑 데이터 분리
                    for row_number, row_data in enumerate(results):
                        self.tableWidget.insertRow(row_number)
                        for column_number, data in enumerate(row_data):
                            self.tableWidget.setItem(row_number, column_number, QTableWidgetItem(str(data)))
            else:
                for query in self.query[query_number].split(';'):
                    if query.strip():
                        self.cursor.execute(query)
                self.conn.commit()
                self.tableWidget.setRowCount(1)
                self.tableWidget.setColumnCount(1)
                self.tableWidget.setItem(0, 0, QTableWidgetItem("쿼리가 성공적으로 실행되었습니다."))
        except pymysql.MySQLError as e:
            self.tableWidget.setRowCount(1)
            self.tableWidget.setColumnCount(1)
            self.tableWidget.setItem(0, 0, QTableWidgetItem(f"오류 발생: {e}"))

    def clear_table(self):
        # 테이블 초기화
        self.tableWidget.clearContents()
        self.tableWidget.setRowCount(0)
        self.tableWidget.setColumnCount(0)
        self.textEdit.clear()

    def closeEvent(self, event):
        # 창이 닫힐 때 데이터베이스 연결 닫기
        self.conn.close()
        event.accept()

if __name__ == "__main__":
    # 애플리케이션 생성
    app = QApplication(sys.argv)

    # 윈도우 인스턴스 생성
    myWindow = WindowClass()

    # 윈도우 표시
    myWindow.show()

    # 애플리케이션 이벤트 루프 실행
    app.exec_()